In [10]:
# CELL 1: Imports & robust paths (works whether notebook is at project root or in AI/notebooks)
import pandas as pd
import numpy as np
from pathlib import Path

# Start from current working directory (where the notebook runs)
BASE = Path.cwd()

# If we don't see raw/ here, but we do see it one level up (typical AI/notebooks layout), go up
if not (BASE / "raw").exists() and (BASE.parent / "raw").exists():
    BASE = BASE.parent

RAW_DIR = BASE / "raw"
PROCESSED_DIR = BASE / "processed"
PROCESSED_DIR.mkdir(exist_ok=True)

print("Working from:", BASE.resolve())
print("RAW_DIR:", RAW_DIR.resolve())
print("PROCESSED_DIR:", PROCESSED_DIR.resolve())

EXPECTED_COLS = [
    "DATE","DAY_OF_THE_WEEK","MONTH","SEASON",
    "ACTIVITY_steps","ACTIVITY_distance","ACTIVITY_soft","ACTIVITY_moderate","ACTIVITY_intense",
    "ACTIVITY_hr_average","ACTIVITY_hr_min","ACTIVITY_hr_max",
    "SLEEP_totalsleeptime","SLEEP_durationtosleep",
    "SLEEP_hr_average","SLEEP_hr_min","SLEEP_hr_max",
    "SLEEP_wakeupcount","SLEEP_wakeupduration",
]


Working from: D:\SOMNiA\AI
RAW_DIR: D:\SOMNiA\AI\raw
PROCESSED_DIR: D:\SOMNiA\AI\processed


In [11]:
# CELL 1.5: Verify we can see your CSVs
csv_files = sorted(RAW_DIR.glob("*.csv"))
print(f"Found {len(csv_files)} CSVs in {RAW_DIR}")
for p in csv_files[:12]:
    print(" -", p.name)


Found 12 CSVs in d:\SOMNiA\AI\raw
 - 1487.csv
 - 2201.csv
 - 2210.csv
 - 3379.csv
 - 4891.csv
 - 5359.csv
 - 5544.csv
 - 6008.csv
 - 6777.csv
 - 7359.csv
 - 8176.csv
 - 9775.csv


In [12]:
# CELL 2: Generic helpers

import pandas as pd
import numpy as np

def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        if c == "DATE":
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def drop_low_quality_rows(df: pd.DataFrame, threshold: float = 0.4) -> pd.DataFrame:
    """Drop rows where more than 'threshold' fraction of non-DATE columns are NaN."""
    non_date = df.drop(columns=["DATE"], errors="ignore")
    frac_nan = non_date.isna().mean(axis=1)
    return df.loc[frac_nan <= threshold].copy()


In [13]:
# CELL 3: Plausibility rules for activity

def enforce_activity_rules(df: pd.DataFrame) -> pd.DataFrame:
    for c in ["ACTIVITY_steps","ACTIVITY_distance","ACTIVITY_soft","ACTIVITY_moderate","ACTIVITY_intense"]:
        if c in df:
            df.loc[df[c] < 0, c] = np.nan

    for c in ["ACTIVITY_hr_min","ACTIVITY_hr_average","ACTIVITY_hr_max"]:
        if c in df:
            df.loc[(df[c] < 30) | (df[c] > 220), c] = np.nan

    trio = {"ACTIVITY_hr_min","ACTIVITY_hr_average","ACTIVITY_hr_max"}
    if trio.issubset(df.columns):
        mask_bad = (
            (df["ACTIVITY_hr_min"] > df["ACTIVITY_hr_average"]) |
            (df["ACTIVITY_hr_average"] > df["ACTIVITY_hr_max"]) |
            (df["ACTIVITY_hr_min"] > df["ACTIVITY_hr_max"])
        )
        df.loc[mask_bad, list(trio)] = np.nan
    return df


In [14]:
# CELL 4: Plausibility rules for sleep

def enforce_sleep_rules(df: pd.DataFrame) -> pd.DataFrame:
    for c in ["SLEEP_totalsleeptime","SLEEP_durationtosleep","SLEEP_wakeupduration"]:
        if c in df:
            df.loc[(df[c] < 0) | (df[c] > 16*3600), c] = np.nan

    for c in ["SLEEP_hr_min","SLEEP_hr_average","SLEEP_hr_max"]:
        if c in df:
            df.loc[(df[c] < 30) | (df[c] > 220), c] = np.nan

    trio = {"SLEEP_hr_min","SLEEP_hr_average","SLEEP_hr_max"}
    if trio.issubset(df.columns):
        mask_bad = (
            (df["SLEEP_hr_min"] > df["SLEEP_hr_average"]) |
            (df["SLEEP_hr_average"] > df["SLEEP_hr_max"]) |
            (df["SLEEP_hr_min"] > df["SLEEP_hr_max"])
        )
        df.loc[mask_bad, list(trio)] = np.nan

    if "SLEEP_wakeupcount" in df:
        df.loc[df["SLEEP_wakeupcount"] < 0, "SLEEP_wakeupcount"] = np.nan
    return df


In [15]:
# CELL 5: Imputation (rolling median → person median → global median) and winsorization

def rolling_then_median_impute(df: pd.DataFrame, window: int = 7) -> pd.DataFrame:
    cols = [c for c in df.columns if c not in ["DATE","person_id"]]
    global_medians = df[cols].median(numeric_only=True)
    for c in cols:
        s = df[c]
        roll = s.rolling(window=window, min_periods=1).median()
        s = s.fillna(roll)
        s = s.fillna(s.median())
        if s.isna().any():
            s = s.fillna(global_medians.get(c, np.nan))
        df[c] = s
    return df

def winsorize_per_person(df: pd.DataFrame, caps=(0.01, 0.99)) -> pd.DataFrame:
    cols = [c for c in df.columns if c not in ["DATE","person_id"]]
    for c in cols:
        q_low, q_hi = df[c].quantile(caps[0]), df[c].quantile(caps[1])
        if pd.isna(q_low) or pd.isna(q_hi):
            continue
        df[c] = df[c].clip(q_low, q_hi)
    return df


In [16]:
# CELL 6: Clean one person's file robustly

def clean_single_file(path: Path) -> tuple[pd.DataFrame, int]:
    """Returns (cleaned_df, rows_dropped)."""
    person_id = path.stem
    df = pd.read_csv(path, encoding="utf-8-sig")  # handle BOMs

    df.columns = [c.strip() for c in df.columns]
    if "DATE" not in df.columns:
        raise ValueError(f"{path.name}: missing DATE column")

    keep = ["DATE"] + [c for c in EXPECTED_COLS if c != "DATE" and c in df.columns]
    df = df[keep].copy()

    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)
    df = df.dropna(subset=["DATE"]).sort_values("DATE").drop_duplicates(subset=["DATE"], keep="last")
    if df.empty:
        raise ValueError(f"{path.name}: no valid DATE rows after parsing.")

    df = coerce_numeric(df)
    df = enforce_activity_rules(df)
    df = enforce_sleep_rules(df)

    before = len(df)
    df = drop_low_quality_rows(df, threshold=0.4)
    dropped = before - len(df)
    if df.empty:
        raise ValueError(f"{path.name}: all rows dropped by quality threshold.")

    df.insert(1, "person_id", person_id)
    df = rolling_then_median_impute(df, window=7)
    df = winsorize_per_person(df, caps=(0.01, 0.99))

    for col in EXPECTED_COLS:
        if col not in df.columns:
            df[col] = np.nan

    ordered = ["DATE","person_id"] + [c for c in EXPECTED_COLS if c != "DATE"]
    df = df[ordered]
    return df, dropped


In [17]:
# CELL 7: Batch over raw/*.csv; save outputs and a QC report (diagnostic + resilient)

import traceback

all_clean = []
qc_rows = []

csv_files = sorted(RAW_DIR.glob("*.csv"))
print(f"Found {len(csv_files)} raw files\n")

for i, csv_path in enumerate(csv_files, 1):
    try:
        cleaned, dropped = clean_single_file(csv_path)
        all_clean.append(cleaned)

        out_path = PROCESSED_DIR / f"{csv_path.stem}_clean.csv"
        cleaned.to_csv(out_path, index=False)

        qc_rows.append({
            "file": csv_path.name,
            "person_id": csv_path.stem,
            "status": "ok",
            "rows_after_clean": len(cleaned),
            "rows_dropped": dropped,
            "na_after_clean_total": int(cleaned.isna().sum().sum()),
            "note": ""
        })
        print(f"{i:>3}/{len(csv_files)} ✅ {csv_path.name} → {out_path.name} (dropped {dropped})")

    except Exception as e:
        tb_first = traceback.format_exc().strip().splitlines()[-1]
        qc_rows.append({
            "file": csv_path.name,
            "person_id": csv_path.stem,
            "status": "error",
            "rows_after_clean": 0,
            "rows_dropped": None,
            "na_after_clean_total": None,
            "note": f"{type(e).__name__}: {e} | {tb_first}"
        })
        print(f"{i:>3}/{len(csv_files)} ❌ {csv_path.name} failed: {e}")

qc = pd.DataFrame(qc_rows)
qc_path = PROCESSED_DIR / "clean_qc_summary.csv"
qc.to_csv(qc_path, index=False)
print(f"\n🧾 Saved QC summary: {qc_path}")

if len(all_clean) > 0:
    merged = pd.concat(all_clean, ignore_index=True).sort_values(["person_id","DATE"])
    merged_path = PROCESSED_DIR / "merged_clean.csv"
    merged.to_csv(merged_path, index=False)
    print(f"📦 Saved merged dataset: {merged_path}")
    display(merged.head())
else:
    print("\n⚠️ No files were cleaned successfully. Open the QC summary to inspect errors:")
    display(qc.head(20))


Found 12 raw files



C:\Users\ryanj\AppData\Local\Temp\ipykernel_18916\2812088600.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)


  1/12 ✅ 1487.csv → 1487_clean.csv (dropped 0)
  2/12 ✅ 2201.csv → 2201_clean.csv (dropped 8)


C:\Users\ryanj\AppData\Local\Temp\ipykernel_18916\2812088600.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)
C:\Users\ryanj\AppData\Local\Temp\ipykernel_18916\2812088600.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)


  3/12 ✅ 2210.csv → 2210_clean.csv (dropped 10)


...

C:\Users\ryanj\AppData\Local\Temp\ipykernel_18916\2812088600.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)


 10/12 ✅ 7359.csv → 7359_clean.csv (dropped 0)


C:\Users\ryanj\AppData\Local\Temp\ipykernel_18916\2812088600.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)
C:\Users\ryanj\AppData\Local\Temp\ipykernel_18916\2812088600.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce", dayfirst=True)


 11/12 ✅ 8176.csv → 8176_clean.csv (dropped 6)
 12/12 ✅ 9775.csv → 9775_clean.csv (dropped 0)

🧾 Saved QC summary: d:\SOMNiA\AI\processed\clean_qc_summary.csv
📦 Saved merged dataset: d:\SOMNiA\AI\processed\merged_clean.csv


,DATE,person_id,DAY_OF_THE_WEEK,MONTH,SEASON,ACTIVITY_steps,ACTIVITY_distance,ACTIVITY_soft,ACTIVITY_moderate,ACTIVITY_intense,ACTIVITY_hr_average,ACTIVITY_hr_min,ACTIVITY_hr_max,SLEEP_totalsleeptime,SLEEP_durationtosleep,SLEEP_hr_average,SLEEP_hr_min,SLEEP_hr_max,SLEEP_wakeupcount,SLEEP_wakeupduration
0,2021-07-06,1487,1,6,0,2584.0,1841.0,16801.0,60.0,0.0,72.0,61.0,114.0,24960.0,120.0,64.0,60.0,71.0,0.0,180.0
1,2021-07-07,1487,2,6,0,4594.0,3201.0,19500.0,600.0,0.0,72.0,61.0,111.0,24540.0,120.0,63.0,59.0,70.0,0.0,120.0
2,2021-07-08,1487,3,6,0,9602.0,7011.0,18242.0,4439.0,0.0,78.0,58.0,148.0,25680.0,120.0,60.0,55.0,66.0,0.0,180.0
3,2021-07-09,1487,4,6,0,14121.0,11017.0,18962.0,4259.0,1500.0,76.0,60.0,110.0,24180.0,120.0,60.0,54.0,68.0,1.0,420.0
4,2021-07-10,1487,5,6,0,7022.0,4835.0,20759.0,661.0,0.0,74.0,53.0,113.0,27120.0,480.0,63.0,59.0,71.0,0.0,840.0


In [19]:
# CELL 8: Sanity checks (load merged if needed)

from pathlib import Path
import pandas as pd
import numpy as np

if "merged" not in globals():
    merged_path = Path("processed/merged_clean.csv")
    if not merged_path.exists():
        raise FileNotFoundError("Run Cell 7 first to produce processed/merged_clean.csv")
    merged = pd.read_csv(merged_path, parse_dates=["DATE"])

qc_path = Path("../processed/clean_qc_summary.csv")
if qc_path.exists():
    qc = pd.read_csv(qc_path)
    display(qc.head(15))
else:
    print("QC summary not found.")

# 1) Non-negative checks
nonneg_cols = [
    "ACTIVITY_steps","ACTIVITY_distance","ACTIVITY_soft","ACTIVITY_moderate","ACTIVITY_intense",
    "SLEEP_totalsleeptime","SLEEP_durationtosleep","SLEEP_wakeupcount","SLEEP_wakeupduration",
]
bad = {c: int((merged[c] < 0).sum()) for c in nonneg_cols if c in merged.columns}
bad

# 2) HR ordering checks
def count_bad_trio(df, prefix):
    trio = [f"{prefix}_hr_min", f"{prefix}_hr_average", f"{prefix}_hr_max"]
    if not all(c in df.columns for c in trio):
        return None
    return int(((df[trio[0]] > df[trio[1]]) |
                (df[trio[1]] > df[trio[2]]) |
                (df[trio[0]] > df[trio[2]])).sum())

{
    "activity_hr_bad_rows": count_bad_trio(merged, "ACTIVITY"),
    "sleep_hr_bad_rows": count_bad_trio(merged, "SLEEP")
}


,file,person_id,status,rows_after_clean,rows_dropped,na_after_clean_total,note
0,1487.csv,1487,ok,290,0,0,NaN
1,2201.csv,2201,ok,282,8,0,NaN
2,2210.csv,2210,ok,280,10,0,NaN
3,3379.csv,3379,ok,286,4,0,NaN
4,4891.csv,4891,ok,290,0,0,NaN
5,5359.csv,5359,ok,289,1,0,NaN
6,5544.csv,5544,ok,283,7,0,NaN
7,6008.csv,6008,ok,288,2,0,NaN
8,6777.csv,6777,ok,285,5,0,NaN
9,7359.csv,7359,ok,290,0,0,NaN


{'activity_hr_bad_rows': 0, 'sleep_hr_bad_rows': 0}